# Laboratorio 9 Data Science

## Nelson  García Bravatti


# Fundamentos de Spark y Python — Laboratorio con DataFrames (Walmart Stock)

**Curso:** Data Science Sección 20

**Duración estimada:** 1.5–2.5 horas

**Modalidad:** Individual (colaboración para dudas conceptuales permitida, entrega individual)

## Objetivos de aprendizaje
Al finalizar, podrá:
1. Iniciar una **SparkSession** y trabajar con **PySpark DataFrames**.
2. **Cargar** un CSV con encabezados e inferencia de tipos.
3. Explorar estructura: **columnas**, **esquema** y **muestras**.
4. Ejecutar **descriptivos** y agregaciones.
5. Aplicar **filtros**, **transformaciones** y **creación de columnas**.
6. Calcular medidas estadísticas (p. ej., **correlación de Pearson**).
7. Realizar consultas **temporales** (día con máximo precio; máximos por año).
8. Comunicar hallazgos de forma ordenada y reproducible.


## Datos
- **Archivo:** `walmart_stock.csv`
- **Periodo:** 2012–2017
- **Columnas típicas:** `Date`, `Open`, `High`, `Low`, `Close`, `Volume`, `Adj Close`

> *Nota:* No modifique el CSV; todas las transformaciones se realizan en el notebook.


## Requisitos previos
- Python 3.9+ (o entorno equivalente en **Google Colab**)
- **Apache Spark 3.x** con PySpark (o `pyspark` preinstalado en Colab)
- Conocimientos básicos de: tipos de datos, funciones de agregación, y uso de notebooks.


## Entregables
1. **Notebook ejecutado** (`.ipynb`) con todas las celdas y salidas visibles.
2. **Conclusiones breves** (5–10 líneas) al final del notebook con interpretaciones clave.
3. Código **comentado** y ordenado.

**Formato de entrega:** Subir `.ipynb` .


## 1) Configuración del entorno
**Opción A — Local:**
```python
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Lab Spark DF — Walmart") \
    .getOrCreate()
print("Spark version:", spark.version)
```

**Opción B — Colab (sugerida si no tiene Spark local):**
```python
!pip -q install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lab Spark DF — Walmart").getOrCreate()
print("Spark version:", spark.version)
```


## 2) Tareas (complete en orden y deje **toda** la evidencia en el notebook)

### 2.1 Inicie una sesión de Spark (si no está iniciada)

In [1]:
!pip -q install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab Spark DF — Walmart").getOrCreate()
print("Spark version:", spark.version)

Spark version: 3.5.1


### 2.2 Cargue el archivo CSV

In [3]:
#Cargar el archivo CSV
file_path = '/content/walmart_stock.csv'
df_stocks = (
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(file_path)
)

### 2.3 ¿Cuáles son los nombres de las columnas?

In [4]:
#Obtener los nombres de las columnas
columns = df_stocks.columns
print("Columnas:", columns)

Columnas: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


### 2.4 Muestre el **esquema** de los datos

In [5]:
#Mostrar el esquema de los datos
df_stocks.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



### 2.5 Muestre las **primeras 5 filas**

In [6]:
#Ver las primeras filas del dataframe
df_stocks.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



### 2.6 Descriptivos con `describe()` + interpretación

In [7]:
from pyspark.sql import functions as F, types as T

#Seleccionar columnas numéricas para describir
numeric_types = (T.DoubleType, T.IntegerType, T.LongType, T.FloatType, T.ShortType, T.DecimalType)
num_cols = [f.name for f in df_stocks.schema.fields if isinstance(f.dataType, numeric_types)]

#Validar que existan columnas numéricas
if not num_cols:
    raise ValueError("No se detectaron columnas numéricas para describir. Revisa el esquema/lectura del CSV.")

#Descriptivos básicos con describe()
desc_basic = df_stocks.select(num_cols).describe()
desc_basic.show(truncate=False)


+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|Open              |High             |Low              |Close            |Volume           |Adj Close        |
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|count  |1258              |1258             |1258             |1258             |1258             |1258             |
|mean   |72.35785375357709 |72.83938807631165|71.9186009594594 |72.38844998012726|8222093.481717011|67.23883848728146|
|stddev |6.76809024470826  |6.768186808159218|6.744075756255496|6.756859163732991|4519780.8431556  |6.722609449996857|
|min    |56.389998999999996|57.060001        |56.299999        |56.419998        |2094900          |50.363689        |
|max    |90.800003         |90.970001        |89.25            |90.470001        |80898100         |84.91421600000001|
+-------+------------------+-----------------+--

*(Escriba aquí sus interpretaciones de al menos dos métricas de `describe()`)*

### Interpretación de métricas:

### 2.7 Máximo y mínimo de `Volume`

### 2.8 ¿Cuántos días tuvieron `Close < 60`?

### 2.9 Crée la columna `Tasa_HV = High/Volume`

### 2.10 ¿Qué porcentaje del tiempo `High > 80`?

### 2.11 Correlación de Pearson entre `High` y `Volume` + interpretación

*(Escriba aquí una interpretación breve del signo y magnitud de la correlación)*

### 2.12 ¿Qué día tuvo el **precio más alto** (`High`)? Devuelva la fila completa

### 2.13 **Media** de la columna `Close`

### 2.14 **Máximo `High` por año**

## 3) Conclusiones (5–10 líneas)

*Escriba aquí sus principales hallazgos e interpretaciones.*

## Buenas prácticas
- Comente bloques no triviales.
- Nombres de variables **claros** (`df_prices`, `max_high_year`, etc.).
- Reutilice resultados intermedios para evitar recalcular.
- Si usa Colab, fije versiones cuando sea necesario.
- Cierre la sesión de Spark al final si corre local: `spark.stop()`.


## Errores comunes
- Olvidar `inferSchema=True` → todo se carga como `string`.
- Mezclar API RDD con DataFrames sin necesidad.
- Usar funciones de Python puras en `withColumn` (use `pyspark.sql.functions`).
- Intentar graficar DataFrames de Spark directamente: primero **convierta** a Pandas con `.toPandas()` en subconjuntos pequeños.


## Rúbrica de evaluación (100 puntos)
**A. Preparación del ambiente** (10 pts)
- (10) SparkSession creada sin errores; versiones y entorno claros.

**B. Carga y documentación de datos** (15 pts)
- (8) CSV cargado con `header` y `inferSchema` correctos.
- (7) Comentarios breves sobre las columnas y supuestos.

**C. Exploración básica** (10 pts)
- (4) Lista de columnas.
- (3) `printSchema()` bien interpretado.
- (3) `show(5)` con observaciones puntuales.

**D. Descriptivos** (10 pts)
- (6) `describe()` ejecutado y leído correctamente.
- (4) Al menos 2 interpretaciones numéricas.

**E. Agregaciones y filtros** (10 pts)
- (5) Máx./mín. de `Volume` correctos.
- (5) Conteo de días con `Close < 60` correcto.

**F. Ingeniería de características** (10 pts)
- (8) Columna `Tasa_HV = High/Volume` correcta y con tipo numérico.
- (2) Justificación breve del indicador.

**G. Métricas estadísticas** (10 pts)
- (7) Correlación `High`–`Volume` calculada.
- (3) Interpretación del valor (signo y magnitud).

**H. Consultas temporales** (10 pts)
- (5) Día con `High` máximo identificado.
- (5) Máximo `High` por año con agrupación y orden correctos.

**I. Comunicación de resultados** (10 pts)
- (6) Conclusiones finales claras y concisas (5–10 líneas).
- (4) Orden, legibilidad y limpieza del notebook.

**J. Estilo y calidad de código** (5 pts)
- (5) Convenciones PEP8 razonables, nombres significativos y ausencia de código muerto.

> **Total: 100 puntos**
